In [1]:
from pyspark.sql import SparkSession



In [2]:
spark = SparkSession\
    .builder\
    .appName('recomedacao')\
    .getOrCreate()
    
spark

23/08/28 12:53:27 WARN Utils: Your hostname, Gabuwp resolves to a loopback address: 127.0.1.1; using 192.168.119.20 instead (on interface eth0)
23/08/28 12:53:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/28 12:53:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/28 12:53:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/08/28 12:53:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
!wget 'https://caelum-online-public.s3.amazonaws.com/challenge-spark/semanas-3-e-4.zip' && unzip semanas-3-e-4.zip -d dados/


--2023-08-28 12:53:35--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semanas-3-e-4.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 52.216.104.131, 52.217.229.17, 52.216.109.11, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|52.216.104.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2588308 (2.5M) [application/zip]
Saving to: ‘semanas-3-e-4.zip’

semanas-3-e-4.zip   100%[===================>]   2.47M  2.38MB/s    in 1.0s    

2023-08-28 12:53:37 (2.38 MB/s) - ‘semanas-3-e-4.zip’ saved [2588308/2588308]

Archive:  semanas-3-e-4.zip
   creating: dados/dataset_ml_parquet/
  inflating: dados/dataset_ml_parquet/_SUCCESS  
  inflating: dados/dataset_ml_parquet/._SUCCESS.crc  
  inflating: dados/dataset_ml_parquet/part-00003-a14b227c-f87e-4893-b5f9-4163ed07cb37-c000.snappy.parquet  
  inflating: dados/dataset_ml_parquet/part-00001-a14b227c-f87e-4893-b5

# Leitura dos dados

In [6]:
dados = spark.read.parquet("dados/dataset_ml_parquet")
dados.show(5)

+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio| iptu|   valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|00002dd9-cc74-480...|    2|       35|        1|      1|   0.0| 0.0|Santo Cristo|     100.0|100.0|245000.0|           1|         0|     

In [7]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: double (nullable = true)
 |-- vaga: double (nullable = true)
 |-- bairro: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- valor: double (nullable = true)
 |-- Zona Central: integer (nullable = true)
 |-- Zona Norte: integer (nullable = true)
 |-- Zona Oeste: integer (nullable = true)
 |-- Zona Sul: integer (nullable = true)
 |-- Academia: integer (nullable = true)
 |-- Animais permitidos: integer (nullable = true)
 |-- Churrasqueira: integer (nullable = true)
 |-- Condomínio fechado: integer (nullable = true)
 |-- Elevador: integer (nullable = true)
 |-- Piscina: integer (nullable = true)
 |-- Playground: integer (nullable = true)
 |-- Portaria 24h: integer (nullable = true)
 |-- Portão eletrônico: integer (nullable 

In [8]:
print(f"Quantidade de linhas: {dados.count()}")
print(f"Quantidade de colunas: {len(dados.columns)}")

Quantidade de linhas: 66551
Quantidade de colunas: 25


In [9]:
import pyspark.sql.functions as f


In [10]:
dados.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in dados.columns]).show()


+---+-----+---------+---------+-------+------+----+------+----------+----+-----+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
| id|andar|area_util|banheiros|quartos|suites|vaga|bairro|condominio|iptu|valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+---+-----+---------+---------+-------+------+----+------+----------+----+-----+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|  0|    0|        0|        0|      0|     0|   0|     0|         0|   0|    0|           0|         0|         0|       0|       0|                 0|            0|                 0|       0|      0|         0|    

# Preparando os dados

In [11]:
from pyspark.ml.feature import VectorAssembler


In [12]:
dados.columns


['id',
 'andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'bairro',
 'condominio',
 'iptu',
 'valor',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul',
 'Academia',
 'Animais permitidos',
 'Churrasqueira',
 'Condomínio fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'Portaria 24h',
 'Portão eletrônico',
 'Salão de festas']

In [13]:
X = dados.columns
X.remove('id')
X.remove('bairro')
X

['andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'condominio',
 'iptu',
 'valor',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul',
 'Academia',
 'Animais permitidos',
 'Churrasqueira',
 'Condomínio fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'Portaria 24h',
 'Portão eletrônico',
 'Salão de festas']

In [14]:
vecAssembler = VectorAssembler(inputCols=X, outputCol='features')

In [15]:
dados_vector = vecAssembler.transform(dados).select(['features'])

In [16]:
dados_vector.show(truncate=False, n=5)

23/08/28 14:09:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+------------------------------------------------------------------------------------------------------------+
|features                                                                                                    |
+------------------------------------------------------------------------------------------------------------+
|[2.0,35.0,1.0,1.0,0.0,0.0,100.0,100.0,245000.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]     |
|(23,[0,1,2,3,5,6,7,8,10,15,17,19,20,22],[1.0,84.0,2.0,2.0,1.0,770.0,105.0,474980.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|(23,[1,2,3,6,7,8,12,14,17],[85.0,2.0,2.0,460.0,661.0,290000.0,1.0,1.0,1.0])                                 |
|(23,[1,2,3,5,6,7,8,11,18,19],[58.0,1.0,2.0,1.0,550.0,550.0,249000.0,1.0,1.0,1.0])                           |
|(23,[1,2,3,4,5,6,8,10],[64.0,2.0,2.0,1.0,1

# Padronizando os dados

In [17]:
from pyspark.ml.feature import StandardScaler


In [18]:
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
scaler_model = scaler.fit(dados_vector)
dados_scaler = scaler_model.transform(dados_vector)

In [19]:
dados_scaler.show(5)

+--------------------+--------------------+
|            features|     scaled_features|
+--------------------+--------------------+
|[2.0,35.0,1.0,1.0...|[0.13607726247524...|
|(23,[0,1,2,3,5,6,...|(23,[0,1,2,3,5,6,...|
|(23,[1,2,3,6,7,8,...|(23,[1,2,3,6,7,8,...|
|(23,[1,2,3,5,6,7,...|(23,[1,2,3,5,6,7,...|
|(23,[1,2,3,4,5,6,...|(23,[1,2,3,4,5,6,...|
+--------------------+--------------------+
only showing top 5 rows



# Redução de dimensionalidade

In [20]:
from pyspark.ml.feature import PCA

In [21]:
pca = PCA(k=2, inputCol='scaled_features', outputCol='pca_features')
model_pca = pca.fit(dados_scaler)
dados_pca = model_pca.transform(dados_scaler)

23/08/28 14:10:58 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/08/28 14:10:58 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


23/08/28 14:10:59 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/08/28 14:10:59 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [22]:
dados_pca.select('pca_features').show(5, truncate=False)

+------------------------------------------+
|pca_features                              |
+------------------------------------------+
|[-6.165125049533941,1.3380985269404202]   |
|[-3.252911181218317,-1.1179591836230234]  |
|[-1.0611769329629646,-1.668504005869305]  |
|[-1.9958119007837098,-1.7655796610848917] |
|[-0.41814060701449185,-2.1310407911416687]|
+------------------------------------------+
only showing top 5 rows



# Extra: Construindo um pipeline

In [23]:
from pyspark.ml import Pipeline

In [24]:
pca_pipeline = Pipeline(stages=[vecAssembler, scaler, pca])


In [25]:
pca_pipeline_model = pca_pipeline.fit(dados)


In [26]:
dados_pca = pca_pipeline_model.transform(dados)

In [27]:
dados_pca.show(5)


+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+--------------------+--------------------+--------------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio| iptu|   valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|            features|     scaled_features|        pca_features|
+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+-----------

In [28]:
dados_pca.select('pca_features').show(truncate=False)

+------------------------------------------+
|pca_features                              |
+------------------------------------------+
|[-6.165125049533941,1.3380985269404195]   |
|[-3.2529111812183165,-1.1179591836230236] |
|[-1.0611769329629643,-1.668504005869304]  |
|[-1.9958119007837098,-1.7655796610848915] |
|[-0.4181406070144919,-2.1310407911416682] |
|[-6.901523864979939,-6.262240376053232]   |
|[-0.7164221492881624,-1.4040232409556406] |
|[-1.4856960209871883,-1.780850707341442]  |
|[-6.045628320891713,-7.824880763294963]   |
|[-6.199240571378156,0.31242092235474406]  |
|[-6.884681111573794,-1.0506751049281162]  |
|[-4.613470387276728,-4.014655897062529]   |
|[-4.4763270618051685,-3.8649262122152934] |
|[-0.9935595784850196,-2.924454860845293]  |
|[-3.164519253343427,-2.145186138172064]   |
|[-3.546059940548771,-4.655748957792847]   |
|[-7.507064348067451,-5.873363966077192]   |
|[-0.6502007974668906,-0.4434774122064886] |
|[-0.12715321773616461,-1.1961148588302148]|
|[-2.84936

# Criando nossos cluster com KMeans

In [29]:
from pyspark.ml.clustering import KMeans


In [30]:
SEED = 1224

In [31]:
kmeans = KMeans(featuresCol='pca_features', predictionCol='cluster_pca', k=5, seed=SEED)


In [32]:
model_kmeans = kmeans.fit(dados_pca)

In [33]:
predictions_kmeans = model_kmeans.transform(dados_pca)


In [34]:
predictions_kmeans.select('pca_features', 'cluster_pca').show(5, truncate=False)


+-----------------------------------------+-----------+
|pca_features                             |cluster_pca|
+-----------------------------------------+-----------+
|[-6.165125049533941,1.3380985269404195]  |2          |
|[-3.2529111812183165,-1.1179591836230236]|3          |
|[-1.0611769329629643,-1.668504005869304] |0          |
|[-1.9958119007837098,-1.7655796610848915]|0          |
|[-0.4181406070144919,-2.1310407911416682]|0          |
+-----------------------------------------+-----------+
only showing top 5 rows



# Extra: Avaliando os clusters

In [35]:
dados_pca\
    .join(predictions_kmeans.select('id', 'cluster_pca'), on='id')\
    .groupBy('cluster_pca')\
    .agg(
        f.count('id').alias('quantidade'),
        f.mean('valor').alias('valor_medio'),
        f.mean('area_util').alias('area_media'),
        f.round(f.mean('quartos'),0).alias('quartos_medio'),
        f.round(f.mean('vaga'), 0).alias('vagas_medio'),
        f.round(f.mean('banheiros'), 0).alias('banheiros_medio'),
        f.round(f.mean('suites'), 0).alias('suites_medio'),
        f.mean('condominio').alias('condominio_medio'),
        f.mean('iptu').alias('iptu_medio'),
    )\
    .orderBy('cluster_pca')\
    .show()

+-----------+----------+-----------------+------------------+-------------+-----------+---------------+------------+------------------+------------------+
|cluster_pca|quantidade|      valor_medio|        area_media|quartos_medio|vagas_medio|banheiros_medio|suites_medio|  condominio_medio|        iptu_medio|
+-----------+----------+-----------------+------------------+-------------+-----------+---------------+------------+------------------+------------------+
|          0|     16435|731467.9844234865| 79.44192272588987|          2.0|        1.0|            2.0|         0.0|1463.4741101308184|1374.2049285062367|
|          1|      9597|2379548.694487861|187.58382827967074|          3.0|        2.0|            4.0|         2.0| 6168.435135979994| 11766.30363655309|
|          2|     20629|878542.7852052936| 91.84948373648747|          3.0|        1.0|            2.0|         1.0| 4347.608754665762| 3676.641184739929|
|          3|     14620|805741.3774281805| 83.46983584131327|         

# Extra: Encontrar o numero ideal de componentes

In [36]:
pca_pipeline_model.stages[2].explainedVariance


DenseVector([0.2655, 0.1721])

In [37]:
# Colunas do conjunto de dados: 
X

['andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'condominio',
 'iptu',
 'valor',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul',
 'Academia',
 'Animais permitidos',
 'Churrasqueira',
 'Condomínio fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'Portaria 24h',
 'Portão eletrônico',
 'Salão de festas']

In [38]:

# Quantidade de colunas existentes
k = len(X)
k

23

In [39]:
dados_pca.select("pca_features").show(5, truncate=False)

+-----------------------------------------+
|pca_features                             |
+-----------------------------------------+
|[-6.165125049533941,1.3380985269404195]  |
|[-3.2529111812183165,-1.1179591836230236]|
|[-1.0611769329629643,-1.668504005869304] |
|[-1.9958119007837098,-1.7655796610848915]|
|[-0.4181406070144919,-2.1310407911416682]|
+-----------------------------------------+
only showing top 5 rows



In [40]:
pca = PCA(k=k, inputCol='scaled_features', outputCol='pca_features')
model_pca = pca.fit(dados_pca.drop('pca_features'))
dados_imoveis_pca = model_pca.transform(dados_pca.drop('pca_features'))

In [41]:
dados_imoveis_pca.select("pca_features").show(5, truncate=False)


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
+---------------------------------------------------------

In [42]:
sum(model_pca.explainedVariance) * 100

99.99999999999997

In [43]:
model_pca.explainedVariance[:]

array([2.65457494e-01, 1.72067301e-01, 9.12744818e-02, 5.43737219e-02,
       5.22298117e-02, 4.66203459e-02, 4.42659008e-02, 4.16021312e-02,
       3.46818214e-02, 2.72266440e-02, 2.43947461e-02, 2.01043117e-02,
       1.92103152e-02, 1.76122901e-02, 1.55070658e-02, 1.38947785e-02,
       1.20074124e-02, 1.13351577e-02, 1.01096341e-02, 9.23012235e-03,
       8.89844650e-03, 7.89606599e-03, 3.47810904e-15])

In [44]:
import numpy as np


In [45]:
lista_valores = np.cumsum(model_pca.explainedVariance[:])
lista_valores

array([0.26545749, 0.43752479, 0.52879928, 0.583173  , 0.63540281,
       0.68202316, 0.72628906, 0.76789119, 0.80257301, 0.82979965,
       0.8541944 , 0.87429871, 0.89350903, 0.91112132, 0.92662838,
       0.94052316, 0.95253057, 0.96386573, 0.97397537, 0.98320549,
       0.99210393, 1.        , 1.        ])

In [46]:
k = sum(lista_valores <= 0.7)
k

6

In [47]:
pca = PCA(k=k, inputCol='scaled_features', outputCol='pca_features')
model_pca = pca.fit(dados_pca.drop('pca_features'))
dados_imoveis_pca_final = model_pca.transform(dados_pca.drop('pca_features'))

In [48]:
dados_imoveis_pca_final.select('pca_features').show(truncate=False, n=5)


+----------------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                                |
+----------------------------------------------------------------------------------------------------------------------------+
|[-6.165125049533941,1.3380985269404189,-1.7052299823815504,-0.5338289630545673,0.08903815478585374,-0.31343961802498577]    |
|[-3.252911181218316,-1.1179591836230232,-0.29238958414996896,3.1955388200208437,0.15286306982185177,1.3028930490659607]     |
|[-1.0611769329629643,-1.668504005869304,-2.307594827839209,0.1055312412577142,-0.06914386452865352,0.7187181899564266]      |
|[-1.9958119007837092,-1.7655796610848904,1.357196249867291,-0.16336519381191897,-0.021305227846933672,0.12808044918263461]  |
|[-0.41814060701449174,-2.1310407911416673,-0.10264099265013303,2.809916415039136,-0.042956284698292005,-0.0944

In [49]:
sum(model_pca.explainedVariance) *100

68.20231562211693

In [50]:
pca_pipeline = Pipeline(stages=[vecAssembler, scaler, pca])


In [51]:
model_pca_pipeline = pca_pipeline.fit(dados)


In [52]:
projection = model_pca_pipeline.transform(dados)

In [53]:
projection.select('pca_features').show(truncate=False, n=5)

+---------------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                               |
+---------------------------------------------------------------------------------------------------------------------------+
|[-6.16512504953394,1.3380985269404186,-1.7052299823815504,-0.5338289630545634,0.08903815478585952,-0.3134396180250046]     |
|[-3.2529111812183165,-1.1179591836230225,-0.2923895841499692,3.1955388200208517,0.1528630698218336,1.3028930490659538]     |
|[-1.0611769329629643,-1.6685040058693046,-2.30759482783921,0.10553124125771662,-0.06914386452865681,0.7187181899564257]    |
|[-1.9958119007837094,-1.7655796610848904,1.3571962498672903,-0.1633651938119181,-0.02130522784693359,0.12808044918263484]  |
|[-0.41814060701449174,-2.1310407911416673,-0.1026409926501338,2.8099164150391385,-0.04295628469830398,-0.094490687673

In [54]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]

evaluator = ClusteringEvaluator(predictionCol='cluster_pca', featuresCol='pca_features', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')
silhouette_score = {}

for i in range(2,51):
    
    KMeans_algo = KMeans(k=i, featuresCol='pca_features', predictionCol='cluster_pca', seed=SEED)
    
    KMeans_fit = KMeans_algo.fit(projection)
    
    output = KMeans_fit.transform(projection)
    
    score = evaluator.evaluate(output)
    silhouette_score[i] = score

In [55]:
silhouette_score


{2: 0.48064469046819147,
 3: 0.35247190341814694,
 4: 0.4111918388989504,
 5: 0.35482021308316913,
 6: 0.4219544176796719,
 7: 0.4723010382878085,
 8: 0.3934593076982329,
 9: 0.49728750051842896,
 10: 0.503364519719561,
 11: 0.5054082634358709,
 12: 0.48659011631495735,
 13: 0.5043544341269074,
 14: 0.48269511538345816,
 15: 0.4932897650231591,
 16: 0.4635348215976478,
 17: 0.508658464779861,
 18: 0.48228397005793516,
 19: 0.5070408969981799,
 20: 0.48690239809849,
 21: 0.4828912380576073,
 22: 0.4963753755298887,
 23: 0.5010154175063846,
 24: 0.5000533667544367,
 25: 0.513485487515551,
 26: 0.48637920764771575,
 27: 0.49142848030669245,
 28: 0.4791224056120226,
 29: 0.46682958045549516,
 30: 0.5143023360040619,
 31: 0.4719678265655038,
 32: 0.484151607897867,
 33: 0.5019622107280337,
 34: 0.47626900095719443,
 35: 0.4771669749482904,
 36: 0.515122693207503,
 37: 0.4737239725118659,
 38: 0.47829720958013283,
 39: 0.520627476060475,
 40: 0.4952133506568015,
 41: 0.49787010486011785,
 42

In [56]:
k_best = [key for key, value in silhouette_score.items() if value == max(silhouette_score.values())]
k_best[0]

39

In [57]:
kmeans = KMeans(k=k_best[0], featuresCol='pca_features', predictionCol='cluster_pca', seed=SEED)


In [58]:
modelo_kmeans = kmeans.fit(projection)

In [59]:
projetion_kmeans = modelo_kmeans.transform(projection)

In [60]:
projetion_kmeans.select(['pca_features','cluster_pca', "id"]).show(truncate=False)


+----------------------------------------------------------------------------------------------------------------------------+-----------+------------------------------------+
|pca_features                                                                                                                |cluster_pca|id                                  |
+----------------------------------------------------------------------------------------------------------------------------+-----------+------------------------------------+
|[-6.16512504953394,1.3380985269404186,-1.7052299823815504,-0.5338289630545634,0.08903815478585952,-0.3134396180250046]      |25         |00002dd9-cc74-4809-b5a5-850adf0e7526|
|[-3.2529111812183165,-1.1179591836230225,-0.2923895841499692,3.1955388200208517,0.1528630698218336,1.3028930490659538]      |24         |0009ca94-2b37-4381-b8b8-773ce0f92444|
|[-1.0611769329629643,-1.6685040058693046,-2.30759482783921,0.10553124125771662,-0.06914386452865681,0.7187181899564257]

In [61]:
id_imovel = '0034df72-124a-4383-a89f-a019850a2ba0'

In [62]:
projetion_kmeans.filter(projetion_kmeans.id == id_imovel).select('cluster_pca').show()

+-----------+
|cluster_pca|
+-----------+
|         10|
+-----------+



In [63]:
cluster = projetion_kmeans.filter(projetion_kmeans.id == id_imovel).select('cluster_pca').collect()[0][0]
cluster


10

In [64]:
imoveis_recomendadas = projetion_kmeans.filter(projetion_kmeans.cluster_pca == cluster)\
                                       .select('bairro', 'id', 'pca_features')
imoveis_recomendadas.show(5)

+--------------------+--------------------+--------------------+
|              bairro|                  id|        pca_features|
+--------------------+--------------------+--------------------+
|Recreio dos Bande...|0034df72-124a-438...|[-4.6134703872767...|
|Recreio dos Bande...|01b872aa-9563-4e6...|[-3.7479325299309...|
|             Taquara|01e94fd8-5fa6-454...|[-2.5093968825908...|
|     Barra da Tijuca|023db788-b55a-428...|[-4.1939429646586...|
|     Barra da Tijuca|030d636c-e083-419...|[-2.8257315052876...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



# Calculando distância euclidiana para encontrar as 10 melhores recomendações

In [65]:
id_imovel

'0034df72-124a-4383-a89f-a019850a2ba0'

In [66]:
imovel_procurado = imoveis_recomendadas.filter(imoveis_recomendadas.id == id_imovel)\
                                          .select('pca_features').collect()[0][0]
imovel_procurado     

DenseVector([-4.6135, -4.0147, 1.3861, 0.2579, -0.0626, 0.3152])

In [67]:
from scipy.spatial.distance import euclidean
from pyspark.sql.types import FloatType

In [68]:
def calculate_euclidean_distance(imovel, valor):
    return euclidean(imovel, valor)

euclidean_udf = f.udf(lambda x: calculate_euclidean_distance(imovel_procurado, x), FloatType())

imoveis_recomendadas\
    .withColumn('distancia', euclidean_udf('pca_features'))\
    .select('bairro', 'id', 'distancia')\
    .orderBy('distancia')\
    .show(5)

+--------------------+--------------------+----------+
|              bairro|                  id| distancia|
+--------------------+--------------------+----------+
|Recreio dos Bande...|0034df72-124a-438...|       0.0|
|Recreio dos Bande...|e5ec2e28-c81f-47a...|0.17295614|
|Recreio dos Bande...|12012273-e197-488...|0.18009204|
|Recreio dos Bande...|c4585166-c411-4e5...|0.18062168|
|         Jacarepaguá|612f2a2c-a28a-44e...|0.24683942|
+--------------------+--------------------+----------+
only showing top 5 rows



In [69]:
recomendadas = imoveis_recomendadas\
    .withColumn('distancia', euclidean_udf('pca_features'))\
    .select('bairro', 'id', 'distancia')\
    .orderBy('distancia')

recomendadas.show(5)

+--------------------+--------------------+----------+
|              bairro|                  id| distancia|
+--------------------+--------------------+----------+
|Recreio dos Bande...|0034df72-124a-438...|       0.0|
|Recreio dos Bande...|e5ec2e28-c81f-47a...|0.17295614|
|Recreio dos Bande...|12012273-e197-488...|0.18009204|
|Recreio dos Bande...|c4585166-c411-4e5...|0.18062168|
|         Jacarepaguá|612f2a2c-a28a-44e...|0.24683942|
+--------------------+--------------------+----------+
only showing top 5 rows



# Criando uma função recomendadora para ser usada como referencia para o time dev

In [70]:
def calculate_euclidean_distance(imovel, valor):
    return euclidean(imovel, valor)


def recommender(id_imovel, dataframe_kmeans):
    cluster = dataframe_kmeans\
        .filter(dataframe_kmeans.id == id_imovel)\
        .select('cluster_pca')\
        .collect()[0][0]

    imoveis_recomendadas = dataframe_kmeans\
        .filter(dataframe_kmeans.cluster_pca == cluster)

    imovel_procurado = imoveis_recomendadas\
        .filter(imoveis_recomendadas.id == id_imovel)\
        .select('pca_features')\
        .collect()[0][0]

    euclidean_udf = f.udf(lambda x: calculate_euclidean_distance(
        imovel_procurado, x), FloatType())

    colunas_nao_utilizadas = [
        'features', 'scaled_features', 'pca_features', 'cluster_pca', 'distancia']

    recomendadas = imoveis_recomendadas\
        .withColumn('distancia', euclidean_udf('pca_features'))\
        .select([col for col in imoveis_recomendadas.columns if col not in colunas_nao_utilizadas])\
        .orderBy('distancia')

    return recomendadas

In [71]:
recommender("0034df72-124a-4383-a89f-a019850a2ba0", projetion_kmeans).select("id", "bairro").show(20, truncate=False)

+------------------------------------+------------------------+
|id                                  |bairro                  |
+------------------------------------+------------------------+
|0034df72-124a-4383-a89f-a019850a2ba0|Recreio dos Bandeirantes|
|e5ec2e28-c81f-47a6-aeff-1288d6773c7b|Recreio dos Bandeirantes|
|12012273-e197-488f-bf81-784cb02ec875|Recreio dos Bandeirantes|
|c4585166-c411-4e57-9095-9b7ac2f5dccc|Recreio dos Bandeirantes|
|dbe16cf4-439c-4812-9233-34507b399d53|Jacarepaguá             |
|612f2a2c-a28a-44ec-b6d1-4d7606932f84|Jacarepaguá             |
|7c3e3897-3734-4a10-bd7f-2efbb28aaef8|Barra da Tijuca         |
|d102b22b-73a1-48d4-b4da-617bfd0e33c6|Recreio dos Bandeirantes|
|4f991db8-b42f-4ac9-97a4-f41d0630ee1e|Recreio dos Bandeirantes|
|8aacfda0-621a-4f9a-8aa6-a97881ac5c0d|Recreio dos Bandeirantes|
|a2fa4e3f-4f7f-4003-809d-a1098d88d687|Recreio dos Bandeirantes|
|5788a47e-04e1-41fd-8d81-dfaeff13d942|Barra da Tijuca         |
|9f12771a-93e0-4fe6-bfd2-02d40e429644|Ba